```
curl 'https://rynekpierwotny.pl/api/v2/offers/offer/13402/?s=offer-detail-map' \
  -H 'authority: rynekpierwotny.pl' \
  -H 'accept: application/json' \
  -H 'content-type: application/json' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36' \
  -H 'referer: https://rynekpierwotny.pl/s/nowe-mieszkania/mapa/' \
  -H 'accept-language: en-US,en;q=0.9,pl-PL;q=0.8,pl;q=0.7' \
  --compressed
```

In [72]:
import requests
import json
import sqlite3
import os
import time

import logging
import logging.handlers

In [99]:
LOG_DIR = 'logs'
DATA_DIR = 'data_rynek_pierwotny'

headers = {
    'authority': 'rynekpierwotny.pl',
    'accept': 'application/json',
    'content-type': 'application/json',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'referer': 'https://rynekpierwotny.pl/s/nowe-mieszkania/mapa/',
    'accept-language': 'en-US,en;q=0.9,pl-PL;q=0.8,pl;q=0.7',
}

root = None

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [131]:
conn = sqlite3.connect('rynek_pierwotny.db')
cur = conn.cursor()

In [71]:
#get the count of tables with the name
cur.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='RYNEK_PIERWOTNY' ''')
cur.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='RYNEK_PIERWOTNY_NOT_FOUND' ''')

#if the count is 1, then table exists
if cur.fetchone()[0]==1 :
	print('Table exists.')
else:
    # Create table
    cur.execute('''CREATE TABLE "RYNEK_PIERWOTNY" (
	"Id"	INTEGER NOT NULL,
	"Address"	TEXT,
	"Construction_date_begin"	TEXT,
	"Construction_date_end"	TEXT,
	"Latitude"	REAL,
	"Longitude"	REAL,
	"Name"	TEXT,
	"Price_type"	INTEGER,
	"Region_full_name"	TEXT,
	"Region_id"	INTEGER,
	"Region_short_name"	TEXT,
	"Stats_distance_from_region_center"	INTEGER,
	"Stats_distance_from_region_region"	TEXT,
	"Stats_distance_from_region_distance"	REAL,
	"Stats_properties_count_for_sale"	INTEGER,
	"Stats_properties_with_roi_count"	INTEGER,
	"Stats_ranges_area_max"	INTEGER,
	"Stats_ranges_area_min"	INTEGER,
	"Stats_ranges_price_m2_min"	INTEGER,
	"Stats_ranges_rooms_max"	INTEGER,
	"Stats_ranges_rooms_min"	INTEGER,
	"Street_name"	TEXT,
	"Street_number"	TEXT,
	"Type"	INTEGER,
	PRIMARY KEY("Id")
);''')
    conn.commit()

In [148]:
#get the count of tables with the name
cur.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='RYNEK_PIERWOTNY_NOT_FOUND' ''')

#if the count is 1, then table exists
if cur.fetchone()[0]==1 :
	print('Table exists.')
else:
    # Create table
    cur.execute('''CREATE TABLE "RYNEK_PIERWOTNY_NOT_FOUND" (
	"Id"	INTEGER NOT NULL
);''')
    conn.commit()

In [151]:
def get_all_fetched_ids():
    results_count = cur.execute("select Id from RYNEK_PIERWOTNY")
    results = results_count.fetchall()
    return [r[0] for r in results]

def get_all_not_found_ids():
    results_count = cur.execute("select Id from RYNEK_PIERWOTNY_NOT_FOUND")
    results = results_count.fetchall()
    return [r[0] for r in results]

def count_data_items(folder):
    return len([name for name in os.listdir(folder)])

def set_handler():
    global root
    index = count_data_items(LOG_DIR)
    handler = logging.handlers.WatchedFileHandler(
        os.environ.get("LOGFILE", f"logs/rynek_pierwotny_{index}.log"))
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    root = logging.getLogger()
    root.setLevel(os.environ.get("LOGLEVEL", "DEBUG"))
    root.addHandler(handler)

def check_if_null(value):
    if value == None:
        return ':null'
    return value

def check_text(value):
    if value:
        value = value.replace('\'', '\'\'')
    return check_if_null(value)

In [152]:
def fetch_rynek_pierwotny(ids: list):
    set_handler()

    # check, which ids where fetched
    fetched_ids = get_all_fetched_ids()
    root.debug(f"Number of ids already fetched: {len(fetched_ids)}")

    not_founnd_ids = get_all_not_found_ids()
    root.debug(f"Number of ids not found: {len(not_founnd_ids)}")

    ids_to_fetch = [item for item in ids if item not in fetched_ids]
    ids_to_fetch = [item for item in ids_to_fetch if item not in not_founnd_ids]
    root.debug(f"Number of ids to fetch: {len(ids_to_fetch)}")

    for id in ids_to_fetch:
        if id not in fetched_ids:
            root.info(f"Fetching offer with id={id}")
            success = False

            while not success:
                try:
                    response = requests.get(f'https://rynekpierwotny.pl/api/v2/offers/offer/{id}/?s=offer-detail-map', headers=headers)
                    json_response = json.loads(response.text)
                    root.debug(f"Got response: {json_response}")
                    success = True
                except Exception as e:
                    root.error("Exception occurred", exc_info=True)
                    time.sleep(30)
                    root.info("Startating again")
                    success = False
            if 'detail' in json_response and json_response['detail'] == 'Nie znaleziono.':
                root.info(f"Id {id} not found.")
                insert_text = f"INSERT OR IGNORE INTO RYNEK_PIERWOTNY_NOT_FOUND VALUES ('{id}')"
            else:
                id = check_if_null(json_response['id'])
                address = check_text(json_response['address'])
                construction_date_lower = check_text(json_response['construction_date_range']['lower'])
                construction_date_upper = check_text(json_response['construction_date_range']['upper'])
                coordinates = json_response['geo_point']['coordinates']
                name = check_text(json_response['name'])
                price_type = check_if_null(json_response['price_type'])
                region_full_name = check_text(json_response['region']['full_name'])
                region_id = check_if_null(json_response['region']['id'])
                region_short_name = check_text(json_response['region']['short_name_reverted'])
                slug = check_if_null(json_response['slug'])
                stats = json_response['stats'] #dict!
                street_name = check_text(json_response['street_name'])
                street_number = check_text(json_response['street_number'])
                type = check_if_null(json_response['type'])
                
                insert_text = f"INSERT OR IGNORE INTO RYNEK_PIERWOTNY VALUES ('{id}', '{address}', '{construction_date_lower}', '{construction_date_upper}', {coordinates[0]}, {coordinates[1]}, '{name}', {price_type}, '{region_full_name}', {region_id}, '{region_short_name}', {stats['distance_from_region']['center']}, '{stats['distance_from_region']['region']}', {stats['distance_from_region']['distance']}, {stats['properties_count_for_sale']}, {stats['properties_with_roi_count']}, {stats['ranges_area_max']}, {stats['ranges_area_min']}, {stats['ranges_price_m2_min']}, {stats['ranges_rooms_max']}, {stats['ranges_rooms_min']}, '{street_name}', '{street_number}', {type})"
                
            try:
                root.debug(insert_text)
                insert_result = cur.execute(insert_text, {'null':None})
                conn.commit()
            except Exception as error:
                root.error("Exception occurred", exc_info=True)
                
            
            root.debug("Waiting 2 seconds")
            time.sleep(2)

In [153]:
ids = list(range(0000,14000))
fetch_rynek_pierwotny(ids)

In [139]:
headers = {
    'authority': 'rynekpierwotny.pl',
    'accept': 'application/json',
    'content-type': 'application/json',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'referer': 'https://rynekpierwotny.pl/s/nowe-mieszkania/mapa/',
    'accept-language': 'en-US,en;q=0.9,pl-PL;q=0.8,pl;q=0.7',
}

id = 14028
response = requests.get(f'https://rynekpierwotny.pl/api/v2/offers/offer/{id}/?s=offer-detail-map', headers=headers)
json_response = json.loads(response.text)
json_response

{'address': 'koszaliński, Gąski, ul. Nadbrzeżna 52C',
 'configuration': {'display_type': 1,
  'hide_properties_prices_for_anonymous_users': False,
  'limited_presentation': False},
 'construction_date_range': {'lower': '2022-03-30',
  'upper': '2024-06-30',
  'bounds': '[]'},
 'description': 'Urokliwy lazur wód Bałtyku oraz piaszczysta plaża tworzą piękną scenerię, którą możesz oglądać przez cały rok. Wystarczy, że wybierzesz Let’s Sea Baltic Park – to miejsce idealne dla Ciebie!\r\n\r\nLet’s Sea to nie tylko mieszkania całoroczne, to także doskonała lokata kapitału jako połączenie własnego lokum na wakacje z zyskami czerpanymi z najmu.\r\n\r\nNasz wyjątkowy kompleks powstaje w miejscu, które łączy bliskość natury z komfortem życia – w miejscowości Gąski gmina Mielno. Wszystko po to, aby każdy mógł cieszyć się pięknem Bałtyku i korzystać z udogodnień całorocznego kompleksu.\r\n\r\nAtuty:\r\n- BEZPOŚREDNIO PRZY PLAŻY\r\nLokalizacja całego kompleksu w pierwszej linii brzegowej oznacza, ż

In [142]:
def check_text(value):
    if value:
        value = value.replace('\'', '\'\'')
    return check_if_null(value)

In [143]:
id = check_if_null(json_response['id'])
address = check_text(json_response['address'])
construction_date_lower = check_text(json_response['construction_date_range']['lower'])
construction_date_upper = check_text(json_response['construction_date_range']['upper'])
coordinates = json_response['geo_point']['coordinates']
name = check_text(json_response['name'])
price_type = check_if_null(json_response['price_type'])
region_full_name = check_text(json_response['region']['full_name'])
region_id = check_if_null(json_response['region']['id'])
region_short_name = check_text(json_response['region']['short_name_reverted'])
slug = check_if_null(json_response['slug'])
stats = json_response['stats'] #dict!
street_name = check_text(json_response['street_name'])
street_number = check_text(json_response['street_number'])
type = check_if_null(json_response['type'])
insert_text = f"INSERT OR IGNORE INTO RYNEK_PIERWOTNY VALUES ('{id}', '{address}', '{construction_date_lower}', '{construction_date_upper}', {coordinates[0]}, {coordinates[1]}, '{name}', {price_type}, '{region_full_name}', {region_id}, '{region_short_name}', {stats['distance_from_region']['center']}, '{stats['distance_from_region']['region']}', {stats['distance_from_region']['distance']}, {stats['properties_count_for_sale']}, {stats['properties_with_roi_count']}, {stats['ranges_area_max']}, {stats['ranges_area_min']}, {stats['ranges_price_m2_min']}, {stats['ranges_rooms_max']}, {stats['ranges_rooms_min']}, '{street_name}', '{street_number}', {type})"
insert_text

"INSERT OR IGNORE INTO RYNEK_PIERWOTNY VALUES ('14028', 'koszaliński, Gąski, ul. Nadbrzeżna 52C', '2022-03-30', '2024-06-30', 15.887953984891748, 54.243783578910076, 'Let''s Sea Baltic Park II', 2, 'zachodniopomorskie, koszaliński, Gąski', 53399, 'Gąski, koszaliński', False, 'Kołobrzegu', 14.5, 130, 0, 70, 26, 0, 3, 1, 'ul. Nadbrzeżna', '52C', 1)"

In [144]:
insert_result = cur.execute(insert_text, {'null':None})
results = insert_result.fetchall()
conn.commit()
results

[]